In [ ]:
##please change the name of the files according to the data used
##keep the dataset file in json.gz format and glove embeddings in the same directory

In [1]:
import os
import json
import gzip
import numpy as np
import pandas as pd
import _pickle as cPickle
import numpy as np 
from sklearn import manifold
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Input, GlobalMaxPooling1D, GlobalMaxPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D
#from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam
import pickle


In [2]:
data = []
with gzip.open('meta_Magazine_Subscriptions.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))

# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])


3385
{'category': ['Magazine Subscriptions', 'Professional & Educational Journals', 'Professional & Trade', 'Humanities & Social Sciences', 'Economics & Economic Theory'], 'tech1': '', 'description': ['REASON is edited for people interested in economic, social, and international issues. Viewpoint stresses individual liberty, private responsibility, and limited government. Some emphasis on Pacific Rim, local/state issues with national impact, science/technology. Regular departments include news/trends, book reviews (mostly history, politics, and economics), and cultural commentary.'], 'fit': '', 'title': '<span class="a-size-medium a-color-secondary"', 'also_buy': ['B002PXVYLE', 'B01MCU84LB', 'B000UHI2LW', 'B01AKS14AQ', 'B002PXVYPU', 'B001THPA26', '1476752842', 'B002CT515Q', 'B00KQ0HP2K', 'B00XZF1JUM', 'B0058EONOM', 'B01FV51RKA', 'B0032KHQTS', 'B002GCU2S0', 'B079JCLNZ4', 'B002PXW18E', 'B00005NIOH', 'B00005N7SD', 'B002LDA9VY', '0345816021', 'B00006KX3K', 'B002GCU2SA', 'B0047VIALE', 'B000

In [3]:
df = pd.DataFrame.from_dict(data)
print(len(df))

3385


In [4]:
df

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,"[Magazine Subscriptions, Professional & Educat...",,[REASON is edited for people interested in eco...,,"<span class=""a-size-medium a-color-secondary""","[B002PXVYLE, B01MCU84LB, B000UHI2LW, B01AKS14A...",[https://images-na.ssl-images-amazon.com/image...,,Reason Magazine,[],[],"[B002PXVYLE, B000UHI2LW, B01MCU84LB, B002PXW18...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B00005N7NQ
1,"[Magazine Subscriptions, Arts, Music &amp; Pho...",,[Written by and for musicians. Covers a variet...,,"<span class=""a-size-medium a-color-secondary""","[B002PXVYGE, B0054LRNC8, B000BVEELE, B00006KC3...",[https://images-na.ssl-images-amazon.com/image...,,String Letter Publishers,[],742 in Magazine Subscriptions (,"[B002PXVYGE, B0054LRNC8, B00006L16A, 171906487...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B00005N7OC
2,"[Magazine Subscriptions, Fashion &amp; Style, ...",,[Allure is the beauty expert. Every issue is f...,,"<span class=""a-size-medium a-color-secondary""","[B001THPA4O, B002PXVZWW, B001THPA1M, B001THPA1...",[https://images-na.ssl-images-amazon.com/image...,,Conde Nast Publications,[],[],"[B002PXVZWW, B001THPA4O, B001THPA1M, B01N819UD...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B00005N7OD
3,"[Magazine Subscriptions, Sports, Recreation & ...",,[FLIGHT JOURNAL includes articles on aviation ...,,"<span class=""a-size-medium a-color-secondary""","[B07JVF7QW4, B00ATQ6FPY, B002G551F6, B00008CGW...",[https://images-na.ssl-images-amazon.com/image...,,AirAge Publishing,[],[],"[B002G551F6, B00ATQ6FPY, B00005N7PT, B001THPA2...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B00005N7O9
4,"[Magazine Subscriptions, Professional & Educat...",,[RIDER is published for the road and street ri...,,"<span class=""a-size-medium a-color-secondary""","[B002PXVYD2, B01BM7TOU6, B000060MKJ, B000BNNIG...",[https://images-na.ssl-images-amazon.com/image...,,EPG Media & Specialty Information,[],[],"[B01BM7TOU6, B000060MKJ, B002PXVYD2, B000BNNIG...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B00005N7O6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3380,[],,"[The game's most widely read publication, GOLF...",,"<span class=""a-size-medium a-color-secondary""",[],[],,TI Golf Holdings,[],[],[],"{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B01HI8V19K
3381,"[Magazine Subscriptions, Sports, Recreation & ...",,"[Built for the true classic car enthusiast, ea...",,"<span class=""a-size-medium a-color-secondary""","[B002PXVYO6, B01FIR6AJ4, B00006KGT1, B01DPP7H5...",[https://images-na.ssl-images-amazon.com/image...,,"Motorsport Marketing, Inc.",[],[],"[B01FV4YCNA, B00006KGT1, B002PXVYO6, B0047VIAI...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B01HI8V1I6
3382,"[Magazine Subscriptions, Fashion & Style]",,[InStyle celebrates the private side of public...,,"<span class=""a-size-medium a-color-secondary""",[],[],,Meredith Corporation,[],[],[],"{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B01HI8V0ZK
3383,[],,[Sports Illustrated brings you spectacular act...,,"<span class=""a-size-medium a-color-secondary""",[],[],,Meredith Corporation,[],[],[],"{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B01HI8V1C2


In [5]:
df.replace(to_replace='',value=np.nan,inplace=True)

In [6]:
df=df.mask(df.applymap(str).eq('[]'))

In [7]:
df.dropna(inplace=True,subset=['description','also_buy','brand','main_cat'])

In [8]:
df.reset_index(drop=True,inplace=True)

In [9]:
df

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,"[Magazine Subscriptions, Professional & Educat...",NaN,[REASON is edited for people interested in eco...,NaN,"<span class=""a-size-medium a-color-secondary""","[B002PXVYLE, B01MCU84LB, B000UHI2LW, B01AKS14A...",[https://images-na.ssl-images-amazon.com/image...,NaN,Reason Magazine,NaN,NaN,"[B002PXVYLE, B000UHI2LW, B01MCU84LB, B002PXW18...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7NQ
1,"[Magazine Subscriptions, Arts, Music &amp; Pho...",NaN,[Written by and for musicians. Covers a variet...,NaN,"<span class=""a-size-medium a-color-secondary""","[B002PXVYGE, B0054LRNC8, B000BVEELE, B00006KC3...",[https://images-na.ssl-images-amazon.com/image...,NaN,String Letter Publishers,NaN,742 in Magazine Subscriptions (,"[B002PXVYGE, B0054LRNC8, B00006L16A, 171906487...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7OC
2,"[Magazine Subscriptions, Fashion &amp; Style, ...",NaN,[Allure is the beauty expert. Every issue is f...,NaN,"<span class=""a-size-medium a-color-secondary""","[B001THPA4O, B002PXVZWW, B001THPA1M, B001THPA1...",[https://images-na.ssl-images-amazon.com/image...,NaN,Conde Nast Publications,NaN,NaN,"[B002PXVZWW, B001THPA4O, B001THPA1M, B01N819UD...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7OD
3,"[Magazine Subscriptions, Sports, Recreation & ...",NaN,[FLIGHT JOURNAL includes articles on aviation ...,NaN,"<span class=""a-size-medium a-color-secondary""","[B07JVF7QW4, B00ATQ6FPY, B002G551F6, B00008CGW...",[https://images-na.ssl-images-amazon.com/image...,NaN,AirAge Publishing,NaN,NaN,"[B002G551F6, B00ATQ6FPY, B00005N7PT, B001THPA2...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7O9
4,"[Magazine Subscriptions, Professional & Educat...",NaN,[RIDER is published for the road and street ri...,NaN,"<span class=""a-size-medium a-color-secondary""","[B002PXVYD2, B01BM7TOU6, B000060MKJ, B000BNNIG...",[https://images-na.ssl-images-amazon.com/image...,NaN,EPG Media & Specialty Information,NaN,NaN,"[B01BM7TOU6, B000060MKJ, B002PXVYD2, B000BNNIG...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7O6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491,"[Magazine Subscriptions, Arts, Music & Photogr...",NaN,[Architectural Digest is the world's foremost ...,NaN,"<span class=""a-size-medium a-color-secondary""","[B0039L2QLS, B001THPA44, B00IT70UDS]",NaN,NaN,Conde Nast,NaN,NaN,NaN,"{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B01F25CP3M
1492,"[Magazine Subscriptions, Fashion & Style]",NaN,"[From the cover to the very last page, Allure ...",NaN,"<span class=""a-size-medium a-color-secondary""","[B001THPA4O, B002PXVZWW, B001THPA1M, B001THPA1...",NaN,NaN,Conde Nast,NaN,NaN,NaN,"{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B01F25DB58
1493,"[Magazine Subscriptions, Fashion & Style, Women]",NaN,[Glamour is for the woman who sets the directi...,NaN,"<span class=""a-size-medium a-color-secondary""","[B001THPA12, B001THPA4O, B002PXVZWW, B001THPA1M]",NaN,NaN,Conde Nast,NaN,NaN,NaN,"{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B01F25DCLQ
1494,"[Magazine Subscriptions, Science, History & Na...",NaN,[Start your incredible journey into the past w...,NaN,"<span class=""a-size-medium a-color-secondary""","[B00005NIOH, B0032KHQTS, B00007AZWJ, B00005N7P...",NaN,NaN,National Geographic Partners LLC,NaN,57 in Magazine Subscriptions (,"[B00005NIOH, B0032KHQTS, B00007AZWJ, B00006KH1...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B01F2MKW0I


In [10]:
df.isnull().sum()

category          54
tech1           1496
description        0
fit             1496
title              0
also_buy           0
image            645
tech2           1496
brand              0
feature         1496
rank             646
also_view        151
details            0
main_cat           0
similar_item    1496
date            1496
price           1491
asin               0
dtype: int64

In [11]:
items=df['asin'].tolist()

In [12]:
y=[]
for i in range(len(items)):
    for j in range(len(df['also_buy'][i])):
        if df['also_buy'][i][j] in items:
            y.append(df['also_buy'][i][j])
    y=", ".join(y)
    df['also_buy'][i]=y
    y=[]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [13]:
df

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,"[Magazine Subscriptions, Professional & Educat...",NaN,[REASON is edited for people interested in eco...,NaN,"<span class=""a-size-medium a-color-secondary""","B0058EONOM, B00005NIOH, B00005N7SD, B00006KX3K...",[https://images-na.ssl-images-amazon.com/image...,NaN,Reason Magazine,NaN,NaN,"[B002PXVYLE, B000UHI2LW, B01MCU84LB, B002PXW18...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7NQ
1,"[Magazine Subscriptions, Arts, Music &amp; Pho...",NaN,[Written by and for musicians. Covers a variet...,NaN,"<span class=""a-size-medium a-color-secondary""","B0054LRNC8, B00S5KZHCS, B000O1PKOG",[https://images-na.ssl-images-amazon.com/image...,NaN,String Letter Publishers,NaN,742 in Magazine Subscriptions (,"[B002PXVYGE, B0054LRNC8, B00006L16A, 171906487...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7OC
2,"[Magazine Subscriptions, Fashion &amp; Style, ...",NaN,[Allure is the beauty expert. Every issue is f...,NaN,"<span class=""a-size-medium a-color-secondary""","B00O85C28Q, B0007INI2C, B0046Z4TZW, B00M9NU5BU...",[https://images-na.ssl-images-amazon.com/image...,NaN,Conde Nast Publications,NaN,NaN,"[B002PXVZWW, B001THPA4O, B001THPA1M, B01N819UD...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7OD
3,"[Magazine Subscriptions, Sports, Recreation & ...",NaN,[FLIGHT JOURNAL includes articles on aviation ...,NaN,"<span class=""a-size-medium a-color-secondary""","B00ATQ6FPY, B00008CGWO, B00007B2N1, B000B7VA4C...",[https://images-na.ssl-images-amazon.com/image...,NaN,AirAge Publishing,NaN,NaN,"[B002G551F6, B00ATQ6FPY, B00005N7PT, B001THPA2...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7O9
4,"[Magazine Subscriptions, Professional & Educat...",NaN,[RIDER is published for the road and street ri...,NaN,"<span class=""a-size-medium a-color-secondary""","B000BNNIGO, B00007AVYH, B0054LRNC8, B00XM0ZLWW...",[https://images-na.ssl-images-amazon.com/image...,NaN,EPG Media & Specialty Information,NaN,NaN,"[B01BM7TOU6, B000060MKJ, B002PXVYD2, B000BNNIG...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7O6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491,"[Magazine Subscriptions, Arts, Music & Photogr...",NaN,[Architectural Digest is the world's foremost ...,NaN,"<span class=""a-size-medium a-color-secondary""",B00IT70UDS,NaN,NaN,Conde Nast,NaN,NaN,NaN,"{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B01F25CP3M
1492,"[Magazine Subscriptions, Fashion & Style]",NaN,"[From the cover to the very last page, Allure ...",NaN,"<span class=""a-size-medium a-color-secondary""",,NaN,NaN,Conde Nast,NaN,NaN,NaN,"{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B01F25DB58
1493,"[Magazine Subscriptions, Fashion & Style, Women]",NaN,[Glamour is for the woman who sets the directi...,NaN,"<span class=""a-size-medium a-color-secondary""",,NaN,NaN,Conde Nast,NaN,NaN,NaN,"{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B01F25DCLQ
1494,"[Magazine Subscriptions, Science, History & Na...",NaN,[Start your incredible journey into the past w...,NaN,"<span class=""a-size-medium a-color-secondary""","B00005NIOH, B00007AZWJ, B00005N7PT, B000FTJ7BE...",NaN,NaN,National Geographic Partners LLC,NaN,57 in Magazine Subscriptions (,"[B00005NIOH, B0032KHQTS, B00007AZWJ, B00006KH1...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B01F2MKW0I


In [14]:
df.replace(to_replace='',value=np.nan,inplace=True)

In [15]:
df.dropna(subset=['also_buy'],inplace=True)

In [16]:
df

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,"[Magazine Subscriptions, Professional & Educat...",NaN,[REASON is edited for people interested in eco...,NaN,"<span class=""a-size-medium a-color-secondary""","B0058EONOM, B00005NIOH, B00005N7SD, B00006KX3K...",[https://images-na.ssl-images-amazon.com/image...,NaN,Reason Magazine,NaN,NaN,"[B002PXVYLE, B000UHI2LW, B01MCU84LB, B002PXW18...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7NQ
1,"[Magazine Subscriptions, Arts, Music &amp; Pho...",NaN,[Written by and for musicians. Covers a variet...,NaN,"<span class=""a-size-medium a-color-secondary""","B0054LRNC8, B00S5KZHCS, B000O1PKOG",[https://images-na.ssl-images-amazon.com/image...,NaN,String Letter Publishers,NaN,742 in Magazine Subscriptions (,"[B002PXVYGE, B0054LRNC8, B00006L16A, 171906487...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7OC
2,"[Magazine Subscriptions, Fashion &amp; Style, ...",NaN,[Allure is the beauty expert. Every issue is f...,NaN,"<span class=""a-size-medium a-color-secondary""","B00O85C28Q, B0007INI2C, B0046Z4TZW, B00M9NU5BU...",[https://images-na.ssl-images-amazon.com/image...,NaN,Conde Nast Publications,NaN,NaN,"[B002PXVZWW, B001THPA4O, B001THPA1M, B01N819UD...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7OD
3,"[Magazine Subscriptions, Sports, Recreation & ...",NaN,[FLIGHT JOURNAL includes articles on aviation ...,NaN,"<span class=""a-size-medium a-color-secondary""","B00ATQ6FPY, B00008CGWO, B00007B2N1, B000B7VA4C...",[https://images-na.ssl-images-amazon.com/image...,NaN,AirAge Publishing,NaN,NaN,"[B002G551F6, B00ATQ6FPY, B00005N7PT, B001THPA2...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7O9
4,"[Magazine Subscriptions, Professional & Educat...",NaN,[RIDER is published for the road and street ri...,NaN,"<span class=""a-size-medium a-color-secondary""","B000BNNIGO, B00007AVYH, B0054LRNC8, B00XM0ZLWW...",[https://images-na.ssl-images-amazon.com/image...,NaN,EPG Media & Specialty Information,NaN,NaN,"[B01BM7TOU6, B000060MKJ, B002PXVYD2, B000BNNIG...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B00005N7O6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488,"[Magazine Subscriptions, Fashion & Style]",NaN,"[At Cond Nast Traveler, we inspire action for ...",NaN,"<span class=""a-size-medium a-color-secondary""",B00007AZWJ,NaN,NaN,Conde Nast,NaN,NaN,NaN,"{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B01DUH14EK
1490,"[Magazine Subscriptions, Fashion & Style, Men]",NaN,[GQ Style curates the best of fashion and luxu...,NaN,"<span class=""a-size-medium a-color-secondary""","B00007AXO2, B0046Z4TZW, B00M9NU5BU, B000UHI61I...",NaN,NaN,Conde Nast,NaN,244 in Magazine Subscriptions (,"[B001THPA3K, B06XWC6PRS, B001TP73UU, B002LDA9V...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B01ET8LMLO
1491,"[Magazine Subscriptions, Arts, Music & Photogr...",NaN,[Architectural Digest is the world's foremost ...,NaN,"<span class=""a-size-medium a-color-secondary""",B00IT70UDS,NaN,NaN,Conde Nast,NaN,NaN,NaN,"{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B01F25CP3M
1494,"[Magazine Subscriptions, Science, History & Na...",NaN,[Start your incredible journey into the past w...,NaN,"<span class=""a-size-medium a-color-secondary""","B00005NIOH, B00007AZWJ, B00005N7PT, B000FTJ7BE...",NaN,NaN,National Geographic Partners LLC,NaN,57 in Magazine Subscriptions (,"[B00005NIOH, B0032KHQTS, B00007AZWJ, B00006KH1...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,NaN,NaN,NaN,B01F2MKW0I


In [17]:
df.to_csv("df.csv")

In [18]:
df=pd.read_csv("df.csv")

In [19]:
for i in range(len(df)):
    df["also_buy"][i]=df["also_buy"][i].split(", ")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
df.to_csv("magazine_df.csv")

In [21]:
df=pd.read_csv("magazine_df.csv")

In [22]:
for i in range(len(df)):
    df["description"][i]="".join(df["description"][i][1:-1].split("','"))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
sub_df=df[df.groupby('brand').brand.transform('count')>3]

In [24]:
sub_df=sub_df.reset_index()

In [25]:
sub_df.to_csv("magazine_df.csv")

In [26]:
##initial cleaning is over here

In [27]:
labels_brand=to_categorical(sub_df["brand"].astype("category").cat.codes)
labels_main_cat=to_categorical(sub_df["main_cat"].astype("category").cat.codes)

In [28]:
labels_brand.shape , labels_main_cat.shape

((609, 68), (609, 1))

In [29]:
docs=sub_df["description"].tolist()

In [30]:
MAX_SEQUENCE_LENGTH = 200
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.2

In [31]:
def unpickle(filename):
    f = open(filename, "rb")
    d = cPickle.load(f)
    f.close()
    return d

glove_data=unpickle("glove_new (1).txt")

In [32]:
embeddings_index=glove_data["embeddings_index"]

In [33]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(docs)
sequences = tokenizer.texts_to_sequences(docs)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)
print('Shape of brand label tensor:', labels_brand.shape)
print('Shape of main category label tensor:', labels_main_cat.shape)

Found 4993 unique tokens.
Shape of data tensor: (609, 200)
Shape of brand label tensor: (609, 68)
Shape of main category label tensor: (609, 1)


In [34]:
indices = np.arange(data.shape[0]) 
np.random.shuffle(indices) 
data = data[indices]
labels_brand = labels_brand[indices]
labels_main_cat = labels_main_cat[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train_brand = labels_brand[:-num_validation_samples]
y_train_main_cat = labels_main_cat[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val_brand = labels_brand[-num_validation_samples:]
y_val_main_cat = labels_main_cat[-num_validation_samples:]

In [35]:
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [36]:
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(75, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(100, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(125, 5, activation='relu')(x)
#x = MaxPooling1D(5)(x)
#x = Conv1D(800, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(100, activation='relu',name="dense")(x)
#two branches
brand = Dense(len(labels_brand[0]), activation='softmax',name="dense_1")(x)
main_category = Dense(len(labels_main_cat[0]), activation='softmax',name="dense_2")(x)

#model = Model(sequence_input, preds)
model = Model(inputs=sequence_input, outputs=[brand, main_category])


Training model.


In [37]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 50)      249700      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 196, 75)      18825       embedding[0][0]                  
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 39, 75)       0           conv1d[0][0]                     
______________________________________________________________________________________________

In [38]:
model.compile(optimizer='rmsprop',
              loss={'dense_1': 'categorical_crossentropy', 'dense_2': 'categorical_crossentropy'},
              loss_weights={'dense_1': 1., 'dense_2': 1.},
              metrics={'dense_1': 'acc', 'dense_2': 'acc'})

In [39]:
model.fit(x_train,
          {'dense_1': y_train_brand, 'dense_2': y_train_main_cat},
          epochs=30, batch_size=64,validation_data=(x_val,{'dense_1': y_val_brand, 'dense_2': y_val_main_cat}))

Epoch 1/30
8/8 [==============================] - 0s 59ms/step - loss: 4.0918 - dense_1_loss: 4.0918 - dense_2_loss: 0.0000e+00 - dense_1_acc: 0.0471 - dense_2_acc: 1.0000 - val_loss: 4.0401 - val_dense_1_loss: 4.0401 - val_dense_2_loss: 0.0000e+00 - val_dense_1_acc: 0.1074 - val_dense_2_acc: 1.0000
Epoch 2/30
8/8 [==============================] - 0s 27ms/step - loss: 3.8030 - dense_1_loss: 3.8030 - dense_2_loss: 0.0000e+00 - dense_1_acc: 0.1189 - dense_2_acc: 1.0000 - val_loss: 3.8823 - val_dense_1_loss: 3.8823 - val_dense_2_loss: 0.0000e+00 - val_dense_1_acc: 0.1240 - val_dense_2_acc: 1.0000
Epoch 3/30
8/8 [==============================] - 0s 28ms/step - loss: 3.5868 - dense_1_loss: 3.5868 - dense_2_loss: 0.0000e+00 - dense_1_acc: 0.1168 - dense_2_acc: 1.0000 - val_loss: 3.8092 - val_dense_1_loss: 3.8092 - val_dense_2_loss: 0.0000e+00 - val_dense_1_acc: 0.1240 - val_dense_2_acc: 1.0000
Epoch 4/30
8/8 [==============================] - 0s 26ms/step - loss: 3.4158 - dense_1_loss: 3.4

In [40]:
from tensorflow.keras.models import model_from_json
model_json = model.to_json()
with open("cnn_model_magazine_30e.json", "w") as json_file:
    json_file.write(model_json)

In [41]:
from tensorflow.keras.models import Model

#model = loaded_model  # include here your original model

layer_name = 'dense'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(data)

In [42]:
intermediate_output.shape

(609, 100)

In [43]:
import pickle

with open("embeddings_magazine.pkl","wb") as f:
    pickle.dump(intermediate_output,f)

In [44]:
#compressing embeddings

In [45]:
from sklearn.decomposition import PCA
pca=PCA(n_components=50)

In [46]:
pca_50d_embeddings=pca.fit_transform(intermediate_output)

In [47]:
pca_50d_embeddings.shape

(609, 50)

In [48]:
import pickle
with open("pca_50d_embeddings_magazine.pkl","wb") as f:
    pickle.dump(pca_50d_embeddings,f)

In [49]:
#now making positive samples

In [50]:
arr = np.empty((0, 3), str)
for i in range(len(sub_df)):
    x=sub_df["also_buy"][i][2:-2].split("', '")
    y=sub_df['asin'][i]
    for j in range(len(x)):
            arr = np.append(arr,np.array([[y,x[j],1]]),axis=0)


In [51]:
with open("positive_samples_magazine.pkl","wb") as f:
    pickle.dump(arr,f)

In [52]:
#now saving 50-d embeddings in our sub_df array

In [53]:
pca_50d_embeddings.shape , sub_df.shape

((609, 50), (609, 21))

In [54]:
emb_df=pd.DataFrame(pca_50d_embeddings)

In [55]:
sub_df['embedding'] = emb_df[emb_df.columns[0:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)

In [56]:
for i in range(len(sub_df)):
    sub_df["embedding"][i]=sub_df['embedding'][i].split(",")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [57]:
#now moving to SIAMESE NETWORK

In [58]:
product_pairs=unpickle("positive_samples_magazine.pkl")

In [59]:
prod_1 = product_pairs[:,0]
prod_2 = product_pairs[:,1]

labels = product_pairs[:,2]

In [60]:
#dataframe is sub_df

In [61]:
docs=prod_1.tolist() + prod_2.tolist()

In [62]:
MAX_NUM_WORDS=570

In [63]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(docs)

seq_prod_1 = tokenizer.texts_to_sequences(prod_1.tolist())
seq_prod_2 = tokenizer.texts_to_sequences(prod_2.tolist())

#sequences = tokenizer.texts_to_sequences(docs)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
#to create sequence of equal length
#data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of prod_1 tensor:', len(seq_prod_1))
print('Shape of prod_2 tensor:', len(seq_prod_2))

Found 569 unique tokens.
Shape of prod_1 tensor: 4668
Shape of prod_2 tensor: 4668


In [64]:
prod_1_input=np.array(seq_prod_1)
prod_2_input=np.array(seq_prod_2)

In [65]:
EMBEDDING_DIM = 50

In [66]:
embeddings_index=dict(zip(sub_df["asin"],sub_df["embedding"]))

In [67]:
# create a embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
print(num_words,EMBEDDING_DIM)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word.upper())
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

570 50


In [68]:
def custom_loss(x,y):
    """
    
    Arguments:
    x -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y -- python list containing three objects:
            x1 -- the encodings for the anchor data
            x2 -- the encodings for the positive data (similar to anchor)
            
    Returns:
    loss -- real number, value of the loss cosine similarity
    """
    print('y.shape (the concat / merge vector) = ',y.shape)
    anchor = y[:,:,0:50]
    positive = y[:,:,50:100]
    
    loss = -(tf.keras.layers.Dot(axes=-1,normalize=True)([anchor, positive]))
    
    #anchor_mag = 
    #anchor_mag = mag_finder(anchor)
    #positive_mag = mag_finder(positive)
    #loss=cosine_distance([anchor,positive])
    #loss.shape
    #loss = dot_pro /(anchor_mag * positive_mag)
    
    return loss

In [69]:
input_1 = Input(name="input_1",shape=(None,), dtype='int32')
input_2 = Input(name="input_2",shape=(None,), dtype='int32')

#common_embed = Embedding(embedding_matrix.shape[0], 50,embeddings_initializer=Constant(embedding_matrix),trainable=True)
common_embed = Embedding(embedding_matrix.shape[0], 50,weights=[embedding_matrix],trainable=True)

#common_embed = Embedding(embedding_matrix.shape[0], 50, embeddings_initializer="uniform",trainable=True)

x1 = common_embed(input_1)
x2 = common_embed(input_2)

merged_vector = Concatenate(axis=-1, name='merged_layer')([x1, x2])

#merged_vector=K.squeeze(merged_vector, axis=1)

model_s = Model(inputs=[input_1, input_2], outputs=merged_vector)

model_s.summary()
opti= Adam(lr=0.0001)

model_s.compile(loss=custom_loss, optimizer=opti, metrics=["acc"])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     28500       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
merged_layer (Concatenate)      (None, None, 100)    0           embedding_1[0][0]          

In [70]:
for i in range(len(prod_1_input)):
    prod_1_input[i] = np.asarray(prod_1_input[i])

for i in range(len(prod_2_input)):
    prod_2_input[i] = np.asarray(prod_2_input[i])

labels =labels.astype(np.int64)

in_1=np.stack(prod_1_input)
in_2=np.stack(prod_2_input)

##checking none 
for i in prod_2_input:
    if i is None:
        print(i)

out=to_categorical(labels)
Y_dummy = np.empty((in_1.shape[0],100))


In [71]:
model_s.fit({'input_1':in_1,'input_2':in_2},y=Y_dummy,epochs=60, batch_size=32)

Epoch 1/60
y.shape (the concat / merge vector) =  (None, 1, 100)
y.shape (the concat / merge vector) =  (None, 1, 100)
146/146 [==============================] - 0s 1ms/step - loss: -0.0446 - acc: 0.1504
Epoch 2/60
146/146 [==============================] - 0s 2ms/step - loss: -0.0662 - acc: 0.1504
Epoch 3/60
146/146 [==============================] - 0s 1ms/step - loss: -0.0681 - acc: 0.1504
Epoch 4/60
146/146 [==============================] - 0s 1ms/step - loss: -0.0703 - acc: 0.1504
Epoch 5/60
146/146 [==============================] - 0s 1ms/step - loss: -0.0728 - acc: 0.1504
Epoch 6/60
146/146 [==============================] - 0s 1ms/step - loss: -0.0757 - acc: 0.1504
Epoch 7/60
146/146 [==============================] - 0s 1ms/step - loss: -0.0789 - acc: 0.1504
Epoch 8/60
146/146 [==============================] - 0s 1ms/step - loss: -0.0824 - acc: 0.1504
Epoch 9/60
146/146 [==============================] - 0s 2ms/step - loss: -0.0862 - acc: 0.1504
Epoch 10/60
146/146 [=======

In [82]:
weights=model_s.layers[2].get_weights()[0]

In [83]:
model_json = model_s.to_json()
with open("magazine_siamese_cosine_model_100e.json", "w") as json_file:
    json_file.write(model_json)

In [84]:
inv_dict = {v: k for k, v in word_index.items()} 

In [85]:
embeddings_dict={}

In [86]:
for i in range(len(inv_dict)):
    if(i==0):
        continue
    embeddings_dict[inv_dict[i]]=weights[i]

In [88]:
import pickle

with open("embeddings from siamese - cosine model magazine 50e.pkl","wb") as f:
    pickle.dump(embeddings_dict,f)